# 5章　リカレントニューラルネットワーク (RNN)
これまで見てきたニューラルネットワークは， **フィードフォワード**（入力信号が一方向だけに伝達されていく）であった．  
一方でフィードフォワードは，時系列データをうまく扱えない（時系列データのパターンを十分に学習することができない）問題がある．  
そのため，本章では **リカレントニューラルネットワーク** (Recurrent Neural Network, RNN) を解説・実装していく．

## 5.1　確率と言語モデル

まずは前章までで扱ったword2vecを復習する．また，言語を確率として扱う「言語モデル」についても解説する．

### 5.1.1　word2vecを確率の視点から眺める

まずはCBOWモデルを復習する．ここでは， $w_1, \cdots, w_T$という単語の列で表されるコーパスを考える．
$t$番目の単語をターゲットとして，その前後の単語（$t-1$番目と$t+1$番目）をコンテキストとして扱う．
図5-1を参照．  
この確率は数式で表すと以下の通り．

$$
P(w_t | w_{t-1}, w_{t+1}) \tag{5.1}
$$

CBOWモデルは(5.1)式の事後確率をモデル化する．

これまでは左右対称のウィンドウを考えてきたが，コンテキストとして左のウィンドウのみに限定することもできる．図5-2を参照．

$$
P(w_t | w_{t-2}, w_{t-1}) \tag{5.2}
$$

ここでは，ハイパーパラメータであるウィンドウサイズについて「左を2単語，右を0単語」と設定した．
この理由は，後ほど説明する「言語モデル」を見越してのことだ．

損失関数は以下の通り．

$$
L = -\log P(w_t | w_{t-2}, w_{t-1}) \tag{5.3}
$$

CBOWモデルの学習で行うことは，式(5.3)で表される損失関数（のコーパス全体での総和）を最小とする重みパラメータを見つけることだ．  
学習の結果，副産物として，単語の意味がエンコードされた「単語の分散表現」が得られる．

では，CBOWモデルの本来の目的であった「コンテキストからターゲットを推測すること」は何かに利用できるだろうか？
そこに「言語モデル」が関係する．

### 5.1.2　言語モデル

**言語モデル** (Language Model) は，単語の並びに対して確率を与える．  
このモデルは，さまざまなアプリケーションで応用可能だ．機械翻訳や音声認識などが，代表例である．  
また，言語モデルは新しい文章を生成する用途にも利用できる（7章を参照）．

以下では，$w_1, \cdots, w_m$の$m$個の単語からなる文章を考える．

同時確率$P(w_1, \cdots, w_m)$は，事後確率を用いて以下のように分解できる．

$$
P(w_1, \cdots, w_m)
= \prod_{t=1}^m P(w_t | w_1, \cdots, w_{t-1}) \tag{5.4}
$$

事後確率のイメージは図5-3を参照．

$P(w_t | w_1, \cdots, w_{t-1})$を表すモデルは， **条件付き言語モデル** (Conditional Language Model) と呼ばれる．
条件付き言語モデルを指して，「言語モデル」と呼ぶ場合も多い．

### 5.1.3　CBOWモデルを言語モデルに？

word2vecのCBOWモデルを，（無理やり）言語モデルに適用するには，以下のような近似を行う．

$$
P(w_1, \cdots, w_m)
= \prod_{t=1}^m P(w_t | w_1, \cdots, w_{t-1})
\sim \prod_{t=1}^m P(w_t | w_{t-2}, w_{t-1}) \tag{5.8}
$$

直前の2つの単語だけに依存して次の単語が決まるモデルであるため，これは「2階マルコフ連鎖」である．

式(5.8)では，コンテキストとして2つの単語を用いたが，コンテキストのサイズは任意の長さに固定できる．
コンテキストが短ければ解けないであろう場面は，図5-4を参照．

では，CBOWモデルのコンテキストのサイズを20や30のように大きくすれば問題は解決するのだろうか？
確かに，CBOWモデルのコンテキストのサイズはいくらでも大きくすることはできる．
しかし，CBOWモデルではコンテキスト内の単語の並びが無視されるという問題がある．

具体例として，図5-5を参照．  
コンテキストの単語の並びも考慮するために中間層において連結を行う手もあるが，パラメータはコンテキストのサイズに比例して増加してしまう．

ここでRNNの出番となる．RNNは，コンテキストがどれだけ長くても，そのコンテキストの情報を記憶するメカニズムを持つ．

## 5.2　RNNとは

RNNにあるRecurrentとは「何度も繰り返し起こること」を意味する．
そのため，RNNは日本語では「循環するニューラルネットワーク」ということになる．
まず初めに，「循環する」という言葉について考えてみよう．

### 5.2.1　循環するニューラルネットワーク

循環するためには「閉じた経路」が必要となる．

RNNの特徴は，ループする経路を持つことだ．データが循環することにより，過去の情報を記憶しながら，最新のデータへと更新される．

RNNで用いられるレイヤを「RNNレイヤ」という名前で呼ぶことにする．
図5-6を参照．

RNNレイヤはループする経路を持つ．このループする経路によって，データがレイヤ内を循環することができる．
なお図5-6では，時刻を$t$として，$x_t$を入力としている．
$x_t$は何らかのベクトルを想定する．

図5-6では「分岐」が起きている．
「分岐」とは，同じものがコピーされて分岐することを意味する．
そして，その分岐した出力のひとつが自分自身への入力となる．

また，以下では紙面の都合上，図5-7のようにレイヤを90度回転させて描画することとする．

### 5.2.2　ループの展開

図5-8に示すように，RNNレイヤのループを展開することで，右方向に伸びる長いニューラルネットワークへと変身させられる．
これは，今まで見てきたフィードフォワード型ニューラルネットワークと同じ構造ではあるが，複数のRNNレイヤがすべて「同じレイヤ」であることが，今までとは異なる．

図5-8にある通り，各時刻のRNNレイヤは，そのレイヤへの入力とひとつ前のレイヤからの出力を受け取る．
数式で書くと以下の通り．

$$
h_t = \tanh(h_{t-1} W_h + x_t W_x + b) \tag{5.9}
$$

RNNでは重みが$W_x, W_h$の2つある．
また，バイアスとして$b$が設定されている．

式(5.9)より，RNNは$h$という「状態」を持っており，式(5.9)の形で更新されると解釈できる．
これが，RNNレイヤは「状態を持つレイヤ」や「メモリ（記憶力）を持つレイヤ」であると言われる所以だ．

RNNの$h$は「状態」を記憶し，時間が1ステップ（1単位）進むに従い，式(5.9)の形で更新される．
本書でも他の文献と同様に，RNNの出力$h$を **隠れ状態** (hidden state) や **隠れ状態ベクトル** (hidden state vector) と呼ぶことにする．

多くの文献では，展開後のRNNレイヤは図5-9の左図のように描かれている．

### 5.2.3　Backpropagation THrough Time

展開後のRNNは横方向に伸びたニューラルネットワークとみなすことができた．
そのためRNNの学習も，通常のニューラルネットワークと同じ手順で実行できる．
図5-10を参照．